In [9]:
import csv
from csv import DictReader
import unicodedata

### Rótulos do dataset

- Uso_solo 
- Município
- Area_m2
- Fid_1
- Last_last
- Cn_mancha1
- Cn_mancha2
- Altitude_mancha1
- Altitude_mancha2
- Dif_altitude
- Comprimento
- Dif_cn
- Valoracao

### Limpeza dos dados

Com base nos rótulos: Area_m2, Fid_1, Last_last. <br>
Foi criado um novo dicionário que irá conter as linhas que não se repetem. <br>
Reduzindo de 7290 linhas para 4248.


In [10]:
# Primeira linha -> Fid_1
# Segunda linha -> Last_last

def sem_repeticao(value, solos_dict):
    for i in solos_dict:
        if value['Area_m2'] == solos_dict[i]['Area_m2']:
            if value['Fid_1'] == solos_dict[i]['Fid_1']:
                if value['Last_last'] == solos_dict[i]['Last_last']:
                    return False
    
    return True


if __name__ == '__main__':
    solos_dict = dict()
    with open('SoilUse_1.csv', encoding='utf8') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        for index, value in enumerate(csv_reader):
            if not solos_dict:
                solos_dict.update({index: value})
            else:
                if sem_repeticao(value, solos_dict):
                    solos_dict.update({index: value})


    print("Numero de linhas: " + str(len(solos_dict)) + "\n")
    
    [print(f'{solo} \n') for solo in list(solos_dict.items())[0:6] ]

Numero de linhas: 4248

(0, {'Uso_solo': 'Pastagem/Campo', 'Municipio': 'Alto Piquiri', 'Area_m2': '25833,24', 'Fid_1': '10173', 'Last_last': '9387', 'Cn_mancha1': '69', 'Cn_mancha2': '55', 'Altitude_mancha1': '283,1219512', 'Altitude_mancha2': '285,3666667', 'Dif_altitude': '2,244715447', 'Comprimento': '206,6017408', 'Dif_cn': '-14', 'Valoracao': '-6492,669664'}) 

(1, {'Uso_solo': 'Floresta Nativa', 'Municipio': 'Alto Piquiri', 'Area_m2': '12947,78', 'Fid_1': '9387', 'Last_last': '9387', 'Cn_mancha1': '69', 'Cn_mancha2': '55', 'Altitude_mancha1': '283,1219512', 'Altitude_mancha2': '285,3666667', 'Dif_altitude': '2,244715447', 'Comprimento': '206,6017408', 'Dif_cn': '-14', 'Valoracao': '-6492,669664'}) 

(2, {'Uso_solo': 'Agricultura Anual', 'Municipio': 'Alto Piquiri', 'Area_m2': '117000000', 'Fid_1': '9505', 'Last_last': '9387', 'Cn_mancha1': '82', 'Cn_mancha2': '55', 'Altitude_mancha1': '352,2107826', 'Altitude_mancha2': '285,3666667', 'Dif_altitude': '66,84411591', 'Comprimento':

#### Implementar terafas

- Adicionar coluna Uso1, Uso2, Municipio_1, Municipio_2, Area_1, Area_2.

- Transformar dicionario num csv com as novas colunas.

- Valoração da fronteira (Vf = [CN(A-B)]*[Alt(A-B)] * Long)

#### Tratamento dos dados

**Passo 1** - Para esta etapa, os dados foram separados em dois dicionários:
- **dicionario_fid_last**, contém **Fid_1** e **Last_last** iguais, <br>
- **dicionario_solos**, contém **Fid_1** e **Last_last** diferentes.

**Passo 2** - Em seguida, será criado outro dicionário que terá novas colunas: <br>
**Uso1, Uso2, Municipio_1, Municipio_2, Area_1, Area_2.**

**Detalhe:** Para o "passo 2", com um dicionário_1 que contém **Fid_1** e **Last_last** iguais, <br>
as novas colunas serão adicionadas com base desse dicionário_1, ou seja, a partir da coluna <br>
**Last_last** do dicionário principal, será consultado no dicionário_1 os dados das novas colunas.

**Passo 3** - Por fim, foi criado uma função que irá adicionar novas chaves e valores no novo <br>
dicionario (**dicionario_principal**). <br>

**IMPORTANTE:** Resultados parciais: <br>
- **dicionario_fid_last:** 1249 <br>
- **dicionario_solos:** 2999


In [11]:

def separar_dicionarios(solos_dict, dicionario_fid_last, dicionario_solos):
    
    for i in solos_dict:
        if solos_dict[i]['Fid_1'] == solos_dict[i]['Last_last']:
            dicionario_fid_last.update({i: solos_dict[i]})
        else:
            dicionario_solos.update({i: solos_dict[i]})
    

def novo_chave_valor(index, dicio_principal, dicio_solo, dicio_fid_last):
    aux_dict = dict()
    
    aux_dict['Uso_solo_1'] = dicio_solo['Uso_solo']
    aux_dict['Uso_solo_2'] = dicio_fid_last['Uso_solo']
    aux_dict['Municipio_1'] = dicio_solo['Municipio']
    aux_dict['Municipio_2'] = dicio_fid_last['Municipio']
    aux_dict['Area_m2_1'] = dicio_solo['Area_m2']
    aux_dict['Area_m2_2'] = dicio_fid_last['Area_m2']
    aux_dict['Fid_1'] = dicio_solo['Fid_1']
    aux_dict['Last_last'] = dicio_solo['Last_last']
    aux_dict['Cn_mancha_1'] = dicio_solo['Cn_mancha1']
    aux_dict['Cn_mancha_2'] = dicio_solo['Cn_mancha2']
    aux_dict['Altitude_mancha_1'] = dicio_solo['Altitude_mancha1']
    aux_dict['Altitude_mancha_2'] = dicio_solo['Altitude_mancha2']
    aux_dict['Dif_altitude'] = dicio_solo['Dif_altitude']
    aux_dict['Comprimento'] = dicio_solo['Comprimento']
    aux_dict['Dif_cn'] = dicio_solo['Dif_cn']
    aux_dict['Valoracao'] = dicio_solo['Valoracao']
    
    dicio_principal.update({index: aux_dict})
    

def unificar_dicionarios(dicionario_principal, dicionario_fid_last, dicionario_solos):
    
    for i in dicionario_solos:
        for j in dicionario_fid_last:
            
            if dicionario_solos[i]['Last_last'] == dicionario_fid_last[j]['Last_last']:
                novo_chave_valor(i, dicionario_principal, dicionario_solos[i], dicionario_fid_last[j])
                break
            

if __name__ == '__main__':
    
    dicionario_solos = dict()
    dicionario_fid_last = dict()
    dicionario_principal = dict()
    
    separar_dicionarios(solos_dict, dicionario_fid_last, dicionario_solos)
    
    unificar_dicionarios(dicionario_principal, dicionario_fid_last, dicionario_solos)
    
#     print(dicionario_principal)
    [print(f'{solo} \n') for solo in list(dicionario_principal.items())[0:6] ]


(0, {'Uso_solo_1': 'Pastagem/Campo', 'Uso_solo_2': 'Floresta Nativa', 'Municipio_1': 'Alto Piquiri', 'Municipio_2': 'Alto Piquiri', 'Area_m2_1': '25833,24', 'Area_m2_2': '12947,78', 'Fid_1': '10173', 'Last_last': '9387', 'Cn_mancha_1': '69', 'Cn_mancha_2': '55', 'Altitude_mancha_1': '283,1219512', 'Altitude_mancha_2': '285,3666667', 'Dif_altitude': '2,244715447', 'Comprimento': '206,6017408', 'Dif_cn': '-14', 'Valoracao': '-6492,669664'}) 

(2, {'Uso_solo_1': 'Agricultura Anual', 'Uso_solo_2': 'Floresta Nativa', 'Municipio_1': 'Alto Piquiri', 'Municipio_2': 'Alto Piquiri', 'Area_m2_1': '117000000', 'Area_m2_2': '12947,78', 'Fid_1': '9505', 'Last_last': '9387', 'Cn_mancha_1': '82', 'Cn_mancha_2': '55', 'Altitude_mancha_1': '352,2107826', 'Altitude_mancha_2': '285,3666667', 'Dif_altitude': '66,84411591', 'Comprimento': '455,8604464', 'Dif_cn': '27', 'Valoracao': '822732,89'}) 

(4, {'Uso_solo_1': 'Floresta Nativa', 'Uso_solo_2': 'Plantios Florestais', 'Municipio_1': 'Alto Piquiri', 'Muni

### Converter para CSV

Após a realização das etapas de filtragem e tratamento dos dados, será criado um novo <br>
arquivo CSV com o novo dicionário. 

- Passo 1: Para isso, vamos precisar das lables do novo arquivo CSV. Então <br>
    adicionamos o parâmetro fieldnames e nele passamos as chaves da primeira <br>
    posição do dicionário.
    
    
- Passo 2: Em seguida, escrevemos no novo arquivo CSV cada linha do novo dicionário <br>
    de solos.


In [12]:

with open('mainSoilUse.csv', mode='w', newline='', encoding='utf8') as file:
    new_file = csv.DictWriter(file, fieldnames=dicionario_principal[0].keys())
    new_file.writeheader()

    for index, dicionario in dicionario_principal.items():
        new_file.writerow(dicionario)

In [13]:
# Confirmando os dados
cont = 0
with open('mainSoilUse.csv', encoding='utf8', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    for i in csv_reader:
        print(f"{i}\n")
        
        if cont < 5: cont += 1
        else: break
        

{'Uso_solo_1': 'Pastagem/Campo', 'Uso_solo_2': 'Floresta Nativa', 'Municipio_1': 'Alto Piquiri', 'Municipio_2': 'Alto Piquiri', 'Area_m2_1': '25833,24', 'Area_m2_2': '12947,78', 'Fid_1': '10173', 'Last_last': '9387', 'Cn_mancha_1': '69', 'Cn_mancha_2': '55', 'Altitude_mancha_1': '283,1219512', 'Altitude_mancha_2': '285,3666667', 'Dif_altitude': '2,244715447', 'Comprimento': '206,6017408', 'Dif_cn': '-14', 'Valoracao': '-6492,669664'}

{'Uso_solo_1': 'Agricultura Anual', 'Uso_solo_2': 'Floresta Nativa', 'Municipio_1': 'Alto Piquiri', 'Municipio_2': 'Alto Piquiri', 'Area_m2_1': '117000000', 'Area_m2_2': '12947,78', 'Fid_1': '9505', 'Last_last': '9387', 'Cn_mancha_1': '82', 'Cn_mancha_2': '55', 'Altitude_mancha_1': '352,2107826', 'Altitude_mancha_2': '285,3666667', 'Dif_altitude': '66,84411591', 'Comprimento': '455,8604464', 'Dif_cn': '27', 'Valoracao': '822732,89'}

{'Uso_solo_1': 'Floresta Nativa', 'Uso_solo_2': 'Plantios Florestais', 'Municipio_1': 'Alto Piquiri', 'Municipio_2': 'Alto 

### Exportar em arquivo GraphML

Para criação do arquivo GraphML, foram realizados os seguintes passos:

- Passo 1: Definir os atributos dos nós e arestas.

- Passo 2: Adicionar em cada vértice os dados das coordenadas.

- Passo 3: Scripts para formatar as linhas de cada nó e aresta, com seus <br>
    dados correspondentes, que serão inseridos no arquivo graphml.

- Passo 4: Criação do arquivo GraphML e inserção das linhas formatadas.

- Passo 5 (adicional): Adição de uma função para definir as cores de cada nó <br>
    a partir do CN.

In [14]:
# Dicionario para adicionar os nós existentes e dados de latitude e longitude
nodes_dict = {}

# Atributos dos nós
node_attributes = [
    ['color', 'string'],
    ['uso_solo', 'string'],
    ['municipio', 'string'],
    ['area', 'double'],
    ['cn', 'int'],
    ['id', 'int'],
    ['altitude', 'double'],
    ['latitude', 'double'],
    ['longitude', 'double'],
]

# Atributos das arestas
edge_attributes = [
    ['dif_cn', 'int'],
    ['dif_altitude', 'double'],
    ['comprimento', 'double'],
    ['weight', 'double'],
    ['color', 'string']
]

In [15]:
mapa_de_cores = {
    "Agricultura Anual": "#FFFB00",
    "Floresta Nativa": "#2BDA00",
    "Pastagem/Campo": "#FF9700",
    "Plantios Florestais": "#FF0000",
    "Solo Exposto/Mineracao": "#C47A14",
    "Varzea": "#0E78FF",
    "Area Urbanizada": "#790FB2"
}

# Retorna o nó ou None
def get_node(id):
    return nodes_dict.get(id, None)

def replace_comma(attr):
    return attr.replace(",", ".")

def add_node(id, attr):
    if get_node(id) is None:
        nodes_dict[id] = attr

# Adiciona dados das coordenadas de cada nó
def add_coords():
    with open('coords.csv', encoding='utf8', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        for i in csv_reader:
            nodes_dict[i['fid_1']]['Latitude'] = replace_comma(i['COORD_X'])
            nodes_dict[i['fid_1']]['Longitude'] = replace_comma(i['COORD_Y'])

# Define a cor de acordo com o CN
def set_color(uso_solo):
    return mapa_de_cores.get(uso_solo, "#000000")

# Checa o peso da aresta, pois caso seja 0, a mesma é descartada 
def check_weight(weight):
    if weight == '0':
        return '0.0001'

    return weight

# Define se a valoracao é menor, igual ou maior que 0
def set_val(weight):
    valoracao = int(float(weight))
    
    if valoracao > 0:
        # return 'Increase'
        return '#F70C3C'
    elif valoracao < 0:
        # return 'Decrease'
        return '#3498DB'
    else:
        # return 'Zero'
        return '#85929E'

def normalize_string(string):
    normalized = unicodedata.normalize('NFD', string)
    return normalized.encode('ascii', 'ignore').decode('utf8')

# Formatação para inserção de um nó
def formatted_node(attr):

    if attr['Fid_1'] == attr['Last_last']:
        return f'''<node id="n{attr['n_id']}">
    <data key="cn">{attr['Cn_mancha2']}</data>
    <data key="color">{set_color(normalize_string(attr['Uso_solo']))}</data>
    <data key="id">{attr['Last_last']}</data>
    <data key="altitude">{replace_comma(attr['Altitude_mancha2'])}</data>
    <data key="area">{replace_comma(attr['Area_m2'])}</data>
    <data key="municipio">{normalize_string(attr['Municipio'])}</data>
    <data key="uso_solo">{normalize_string(attr['Uso_solo'])}</data>
    <data key="latitude">{attr['Latitude']}</data>
    <data key="longitude">{attr['Longitude']}</data>
</node>
'''
    else:

        return f'''<node id="n{attr['n_id']}">
    <data key="cn">{attr['Cn_mancha1']}</data>
    <data key="color">{set_color(normalize_string(attr['Uso_solo']))}</data>
    <data key="id">{attr['Fid_1']}</data>
    <data key="altitude">{replace_comma(attr['Altitude_mancha1'])}</data>
    <data key="area">{replace_comma(attr['Area_m2'])}</data>
    <data key="municipio">{normalize_string(attr['Municipio'])}</data>
    <data key="uso_solo">{normalize_string(attr['Uso_solo'])}</data>
    <data key="latitude">{attr['Latitude']}</data>
    <data key="longitude">{attr['Longitude']}</data>
</node>
'''

def get_source_target(solo):
    node_1 = get_node(solo['Fid_1'])
    node_2 = get_node(solo['Last_last'])

    if node_1['Fid_1'] == node_1['Last_last'] and node_2['Fid_1'] == node_2['Last_last']:
        if node_1['Altitude_mancha2'] > node_2['Altitude_mancha2']:
            source = node_1['n_id']
            target = node_2['n_id']
        else:
            source = node_2['n_id']
            target = node_1['n_id']

    elif node_1['Fid_1'] == node_1['Last_last']:
        if node_1['Altitude_mancha2'] > node_2['Altitude_mancha1']:
            source = node_1['n_id']
            target = node_2['n_id']
        else:
            source = node_2['n_id']
            target = node_1['n_id']

    elif node_2['Fid_1'] == node_2['Last_last']:
        if node_1['Altitude_mancha1'] > node_2['Altitude_mancha2']:
            source = node_1['n_id']
            target = node_2['n_id']
        else:
            source = node_2['n_id']
            target = node_1['n_id']

    else:

        if node_1['Altitude_mancha1'] > node_2['Altitude_mancha1']:
            source = node_1['n_id']
            target = node_2['n_id']
        else:
            source = node_2['n_id']
            target = node_1['n_id']
    
    return source, target


# Formatação para inserção de uma aresta
# Peso e Valoracao sao atributos com mesmos valores para melhor
# visualizacao no Gephi
def formatted_edge(edge_id, solo):

    weight = replace_comma(solo['Valoracao'])
    source, target = get_source_target(solo)
    

    return f'''<edge id="e{edge_id}" source="n{source}" target="n{target}">
    <data key="comprimento">{replace_comma(solo['Comprimento'])}</data>
    <data key="dif_altitude">{replace_comma(solo['Dif_altitude'])}</data>
    <data key="dif_cn">{solo['Dif_cn']}</data>
    <data key="weight">{check_weight(weight)}</data>
    <data key="color">{set_val(weight)}</data>
</edge>
'''
    

In [16]:
graphml_file = 'mainSoilUse.graphml'

with open(graphml_file, 'w') as file:

    # Escreva o cabeçalho do arquivo GraphML
    file.write('''<?xml version="1.0" encoding="UTF-8"?>\n<graphml xmlns="http://graphml.graphdrawing.org/xmlns" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://graphml.graphdrawing.org/xmlns http://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd">''')

    # Adicionar atributos de nos
    for node_attr in node_attributes:
        file.write(f'<key id="{node_attr[0]}" for="node" attr.name="{node_attr[0]}" attr.type="{node_attr[1]}"/>\n')
    
    # Adicionar atributos de arestas
    for edge_attr in edge_attributes:
        file.write(f'<key id="{edge_attr[0]}" for="edge" attr.name="{edge_attr[0]}" attr.type="{edge_attr[1]}"/>\n')


    # Iniciando grafo
    file.write('<graph id="G" edgedefault="directed">\n')
    
    # Adicionar nós num dicionario
    for index, dicionario in solos_dict.items():
        add_node(dicionario['Fid_1'], dicionario)
    
    # Adicionar as coordenadas no dicionario dos nós
    add_coords()

    # Adiciona no arquivo o nó formatado
    node_id = 0
    for fid, attr in nodes_dict.items():
        attr['n_id'] = node_id
        file.write(formatted_node(attr))
        
        node_id += 1


    edge_id = 0
    for i, solo in dicionario_principal.items():
        file.write(formatted_edge(edge_id, solo))
        edge_id += 1


    file.write('</graph>\n')
    file.write('</graphml>\n')

##### Aplicando algoritmos no Gephi

Com o algoritmo BFS desenvolvido, o próximo passo será importá-lo no Gephi. Utilizando o plugin Scripting Tool do software, será possível acessar um terminal baseado em Jython (que fornece implementações de Python em Java), no qual serão executados os seguintes comandos:
  - execfile("\<path\>\\\\tcc.py") - Windows
  - execfile("\<path\>/tcc.py") - Linux

**Atenção**: `<path>` é todo o caminho até onde o arquivo `tcc.py` está